In [40]:
import numpy as np
from tensorflow import keras
from keras.models import Model
from keras.layers import Conv2D, Dense, BatchNormalization
from keras.layers import Flatten, MaxPooling2D, Activation
from keras.layers import InputLayer, Input, SpatialDropout2D 
from keras.layers import RandomFlip, RandomZoom, RandomRotation
from sklearn.model_selection import train_test_split

In [41]:
data = np.load('flatland_train.npz')
X = data['X']
y = data['y'].astype(int)

y[y != 0] -= 2    # Correct labels so that triangle is mapped to class 1
X = X / 255.      # Scale down to range [0, 1]

In [42]:
y_true = y.copy()
input = np.array([19, 399, 436, 473, 505, 563, 597, 619, 679, 696, 756, 760, 
                  1238, 1403, 1409, 1413, 1442, 1467, 1474, 1598, 1748, 1854,
                  1975, 2105, 2148, 2183, 2225, 2226, 2281, 2345, 2419, 2601,
                  2676, 2721, 2748, 2752, 2813, 2978, 2980, 3029, 3092, 3110,
                  3143, 3249, 3276, 3357, 3431, 3441, 3798, 3802, 3809, 3853, 
                  3894, 3896, 3988, 4006, 4025, 4037, 4063, 4068, 4094, 4115,
                  4168, 4243, 4273, 4294, 4402, 4452, 4484, 4488, 4645, 4659,
                  4759, 4804, 4816, 4907, 4980, 5034, 5045, 5056, 5061, 5083,
                  5130, 5333, 5375, 5423, 5515, 5571, 5846, 5859, 5887, 5909,
                  5965, 5996, 6006, 6045, 6068, 6131, 6155, 6162, 6222, 6317,
                  6376, 6384, 6452, 6517, 6559, 6653, 6670, 6674, 6679, 6776,
                  6942, 7008, 7093, 7107, 7129, 7220, 7338, 7490, 7557, 7575,
                  7601, 7655, 7806, 7908, 7956, 7983, 8068, 8189, 8360, 8585,
                  8613, 8631, 8635, 8724, 8919, 9089, 9197, 9344, 9495, 9566,
                  9591, 9664, 9687, 9704, 9718, 9839])
output = np.array([4, 4, 4, 5, 5, 4, 3, 4, 6, 0, 5, 0, 3, 3, 6, 4, 3, 6, 3, 0, 
                   4, 5, 4, 3, 4, 5, 5, 6, 0, 3, 5, 5, 3, 6, 5, 6, 6, 3, 5, 0, 
                   6, 0, 5, 6, 4, 4, 3, 6, 3, 6, 4, 0, 5, 5, 3, 4, 5, 6, 3, 4,
                   5, 3, 0, 3, 6, 6, 0, 5, 3, 6, 5, 3, 5, 4, 3, 5, 3, 5, 3, 3,
                   0, 3, 3, 6, 5, 6, 4, 3, 5, 5, 0, 3, 0, 5, 6, 5, 5, 4, 0, 6, 
                   3, 4, 4, 4, 0, 4, 5, 6, 3, 0, 3, 5, 3, 0, 0, 3, 4, 3, 0, 0,
                   6, 6, 0, 0, 3, 6, 3, 0, 4, 4, 4, 6, 3, 6, 0, 3, 6, 5, 0, 5,
                   0, 3, 4, 3, 3, 4, 0, 6])
output[output != 0] -= 2

y_true[input] = output

In [45]:
inp = Input(shape=(50, 50, 1))

# Random augmentation layers
proc = RandomRotation(factor=0.5, fill_mode='constant')(inp)
proc = RandomZoom((-0.3, 0.3), (-0.3, 0.3), fill_mode='constant')(proc)

# Convolutional layers
out = Conv2D(64, kernel_size=(7, 7), strides=2, activation='relu')(proc)
out = BatchNormalization()(out)
out = SpatialDropout2D(0.2)(out)

out = Conv2D(128, kernel_size=(5, 5), activation='relu')(out)
out = BatchNormalization()(out)
out = SpatialDropout2D(0.2)(out)

out = MaxPooling2D(pool_size=(3, 3))(out)

out = Conv2D(64, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = SpatialDropout2D(0.1)(out)

out = Conv2D(32, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)

# Dense layers
out = Flatten()(out)
out = Dense(512, activation="relu")(out)
out = keras.layers.Dropout(0.5)(out)
out = Dense(5, activation="softmax")(out)

model = Model(inputs=inp, outputs=out)
test_model = Model(inputs=proc, outputs=out)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])
test_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])

In [47]:
model.fit(X, y_true, batch_size=16, epochs=20)

Epoch 1/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0412 - accuracy: 0.9861
Epoch 2/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0319 - accuracy: 0.9897
Epoch 3/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0439 - accuracy: 0.9838
Epoch 4/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0313 - accuracy: 0.9896
Epoch 5/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0335 - accuracy: 0.9892
Epoch 6/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0390 - accuracy: 0.9871
Epoch 7/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0319 - accuracy: 0.9886
Epoch 8/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0335 - accuracy: 0.9875
Epoch 9/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0311 - accuracy: 0.9889
Epoch 10/20
625/625 [==============================] - 3s 5ms/step - loss: 0.0368 - accuracy: 0.9879

In [48]:
# Have to use a model without the augmentation layers, because the evaluation 
# site has older version of TensorFlow
test_model.save('good_model.h5')